In [1]:
import os
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import glob 
import pytz
import dask.array as da

In [2]:
# Set the folder path where your .nc files are located
folder_path = '/home/paulc600/local/Easymore/HYPE_merged_easymore/*.nc'

In [3]:
# Set the output folder path where you want to save the text files
output_folder_path = '/home/paulc600/scratch/HYPE/Landsat/01_inputs/'

In [4]:
# Get a list of all .nc files in the folder
file_list = glob.glob(folder_path)

In [5]:
# Open the .nc files using xarray's open_mfdataset
dataset = xr.open_mfdataset(file_list, combine='by_coords')


In [6]:
dataset

<xarray.Dataset>
Dimensions:        (time: 341496, ID: 473)
Coordinates:
  * time           (time) datetime64[ns] 1980-01-01T13:00:00 ... 2018-12-31T1...
  * ID             (ID) float64 1.136e+05 1.136e+05 ... 1.144e+05 1.144e+05
Data variables:
    latitude       (time, ID) float64 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    longitude      (time, ID) float64 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    precipitation  (time, ID) float32 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    temperature    (time, ID) float32 dask.array<chunksize=(8760, 473), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.0.5 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    Author:       The data were written by author name
    History:      Created Thu Jun 22 10:58:17 2023
    history:      Tue Jun 27 00:26:07 2023: cdo mergetime SMMFullTime_easymor...
    CDO:          Climate Data Operators version 2.0.5 (https://mpimet.mpg.de...

In [7]:
# Extract the time coordinate from the dataset
time_coords = dataset.coords['time']

In [8]:
# Convert the object array to pandas Timestamp objects
time_values = pd.to_datetime(time_coords.data)

In [9]:
# Define the UTC and MST time zones
utc_tz = pytz.UTC
mst_tz = pytz.timezone('US/Mountain')

In [10]:
# Convert time from UTC to MST using pandas Timestamp
converted_times = [utc_tz.localize(dt).astimezone(mst_tz) for dt in time_values]

In [11]:
# Create a new dataset with the converted time coordinate
ds_converted = dataset.assign_coords(time=converted_times)

In [12]:
ds_converted

<xarray.Dataset>
Dimensions:        (time: 341496, ID: 473)
Coordinates:
  * time           (time) object 1980-01-01T06:00:00-07:00 ... 2018-12-31T05:...
  * ID             (ID) float64 1.136e+05 1.136e+05 ... 1.144e+05 1.144e+05
Data variables:
    latitude       (time, ID) float64 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    longitude      (time, ID) float64 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    precipitation  (time, ID) float32 dask.array<chunksize=(8760, 473), meta=np.ndarray>
    temperature    (time, ID) float32 dask.array<chunksize=(8760, 473), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.0.5 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    Author:       The data were written by author name
    History:      Created Thu Jun 22 10:58:17 2023
    history:      Tue Jun 27 00:26:07 2023: cdo mergetime SMMFullTime_easymor...
    CDO:          Climate Data Operators version 2.0.5 (https://mpimet.mpg.de...

In [13]:
# Choose a specific time index and ID index for comparison
time_idx = 10092  # Replace with the desired time index
id_idx = 298    # Replace with the desired ID index

In [14]:
original_temperature = dataset['precipitation'].isel(time=time_idx, ID=id_idx).compute()  # Compute Dask array to get actual values
converted_temperature = ds_converted['precipitation'].isel(time=time_idx, ID=id_idx).compute()  # Compute Dask array to get actual values

In [15]:
ds_converted.coords

Coordinates:
  * time     (time) object 1980-01-01T06:00:00-07:00 ... 2018-12-31T05:00:00-...
  * ID       (ID) float64 1.136e+05 1.136e+05 1.136e+05 ... 1.144e+05 1.144e+05

In [16]:
# Print the data values for comparison
print("Original Temperature:", original_temperature)
print("Converted Temperature:", converted_temperature)

Original Temperature: <xarray.DataArray 'precipitation' ()>
array(0.)
Coordinates:
    time     datetime64[ns] 1981-02-26T01:00:00
    ID       float64 1.142e+05
Attributes:
    long_name:  Analysis: Quantity of precipitation
    units:      m
Converted Temperature: <xarray.DataArray 'precipitation' ()>
array(0.)
Coordinates:
    time     datetime64[ns] 1981-02-26T01:00:00
    ID       float64 1.142e+05
Attributes:
    long_name:  Analysis: Quantity of precipitation
    units:      m


In [17]:
# Resample 'temperature' to daily mean and 'precipitation' to daily sum
temperature_daily_mean = ds_converted['temperature'].resample(time='D').mean(dim='time')
precipitation_daily_sum = ds_converted['precipitation'].resample(time='D').sum(dim='time')

In [18]:
temperature_daily_mean

In [19]:
# Convert the resampled data variables to pandas DataFrames
temperature_df = temperature_daily_mean.to_dataframe()
precipitation_df = precipitation_daily_sum.to_dataframe()

In [20]:
# read shapefile with extra river ID given to catchments without rivers
New_ID=gpd.read_file('/home/paulc600/local/SMM/Modified_HYPE/Modified_SMMcat.shp')

In [21]:
# Reset the index of the DataFrames
precipitation_df = precipitation_df.reset_index()
temperature_df = temperature_df.reset_index()

In [23]:
# Convert ID column to integer
precipitation_df['ID'] = precipitation_df['ID'].astype(int)
temperature_df['ID'] = temperature_df['ID'].astype(int)
New_ID['seg_nhm'] = New_ID['seg_nhm'].astype(int)

In [24]:
# Format precipitation and temperature data to 3 significant figures
precipitation_df['precipitation'] = precipitation_df['precipitation'].apply(lambda x: '{:.3g}'.format(x) if not np.isnan(x) else '')
temperature_df['temperature'] = temperature_df['temperature'].apply(lambda x: '{:.3g}'.format(x) if not np.isnan(x) else '')

In [25]:
temperature_df

In [26]:
temperature_pivoted = temperature_df.pivot(index='time', columns='ID', values='temperature')
precipitation_pivoted = precipitation_df.pivot(index='time', columns='ID', values='precipitation')

In [27]:
precipitation_pivoted

ID,113490,113492,113494,113496,113497,113498,113500,113501,113503,113506,...,114432,114435,114437,114442,114445,114447,114450,114455,114458,114460
time,,,,,,,,,,,,,,,,,,,,,
1980-01-01 00:00:00-07:00,0.000754,0.000938,0.000873,0.000344,0.000113,0.000604,5.94e-05,0.000345,5.55e-05,0.000201,...,0.000131,0.000122,0.00011,0.000109,0,0,4.67e-09,2.65e-06,3.24e-06,6.27e-07
1980-01-02 00:00:00-07:00,0.000651,0.000967,0.000936,0.000403,0.000193,0.00103,0.000141,0.000282,0.000102,0.000169,...,0.000798,0.000894,0.000836,0.000692,0.000136,1.21e-05,8.81e-06,7.62e-05,8.66e-05,4.76e-05
1980-01-03 00:00:00-07:00,0.00444,0.00517,0.00477,0.00105,0.000325,0.00327,6.59e-05,0.00114,4.13e-05,0.000662,...,0.00146,0.00135,0.00152,0.00219,0.000361,0.000158,6.35e-05,7.85e-05,0.000139,0.000268
1980-01-04 00:00:00-07:00,0.000467,0.000369,0.00034,0.000198,0.000141,0.000175,7.88e-05,0.000211,7.39e-05,0.00014,...,0.00192,0.00255,0.00239,0.00205,0.000477,0.000411,0.000214,0.0002,0.000142,0.000136
1980-01-05 00:00:00-07:00,0.017,0.0176,0.0174,0.0148,0.0123,0.017,0.0119,0.0147,0.0118,0.0134,...,0.0111,0.0115,0.0106,0.00959,0.00608,0.00462,0.0035,0.0034,0.00359,0.00477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27 00:00:00-07:00,0.000664,0.000655,0.000654,0.000476,0.0004,0.000745,0.000349,0.000438,0.000339,0.000396,...,0.000338,0.00031,0.000311,0.000302,0.000281,0.000455,0.000474,0.000187,0.000196,0.000232
2018-12-28 00:00:00-07:00,0.000418,0.00042,0.000401,0.000253,0.000141,0.000312,0.000116,0.00026,0.000116,0.000195,...,0.000216,0.00024,0.000215,0.00024,4.55e-05,2.93e-05,5.02e-05,7.67e-05,5.77e-05,2.64e-05
2018-12-29 00:00:00-07:00,0.0132,0.0178,0.017,0.00817,0.00262,0.0149,0.00195,0.00748,0.0019,0.00466,...,0.000144,0.000139,0.000192,0.000225,3.25e-06,5.74e-06,5.49e-06,0,1.36e-07,1.76e-06


In [28]:
New_ID.set_index('hru_nhm').loc[precipitation_pivoted.columns, 'seg_nhm']

ID
113490    58219
113492    58241
113494    58237
113496    58232
113497    58214
          ...  
114447    58610
114450    58389
114455    58570
114458    58579
114460    58465
Name: seg_nhm, Length: 473, dtype: int64

In [27]:
# replace ID with river ID from modified shapefile (including imaginary rivers)
precipitation_pivoted.columns=New_ID.set_index('hru_nhm').loc[precipitation_pivoted.columns, 'seg_nhm']
temperature_pivoted.columns=New_ID.set_index('hru_nhm').loc[temperature_pivoted.columns, 'seg_nhm']

In [29]:
# Convert the datetime index to a string representation with 'YYYY-MM-DD' format
temperature_pivoted.index = temperature_pivoted.index.strftime('%Y-%m-%d')
precipitation_pivoted.index = precipitation_pivoted.index.strftime('%Y-%m-%d')

In [30]:
print(temperature_pivoted.index)

Index(['1980-01-01', '1980-01-02', '1980-01-03', '1980-01-04', '1980-01-05',
       '1980-01-06', '1980-01-07', '1980-01-08', '1980-01-09', '1980-01-10',
       ...
       '2018-12-22', '2018-12-23', '2018-12-24', '2018-12-25', '2018-12-26',
       '2018-12-27', '2018-12-28', '2018-12-29', '2018-12-30', '2018-12-31'],
      dtype='object', name='time', length=14245)


In [31]:
temperature_pivoted

seg_nhm,58219,58241,58237,58232,58214,58233,58213,58223,58221,58216,...,58448,58380,58342,58582,58405,58610,58389,58570,58579,58465
time,,,,,,,,,,,,,,,,,,,,,
1980-01-01,-3.11,-3.44,-3.44,-3.53,-2.87,-3.37,-2.94,-3.51,-2.87,-3.22,...,-6.77,-6.75,-6.7,-6.4,-6.22,-8.7,-8.94,-9.06,-7.82,-7.62
1980-01-02,-4.28,-4.19,-4.25,-5.78,-5.89,-4.11,-6.24,-5.69,-5.87,-5.66,...,-8.14,-8.27,-8.32,-9.06,-7.59,-8.35,-8.31,-8.65,-9.22,-8.56
1980-01-03,-6.21,-6.51,-6.53,-6.67,-5.52,-6.5,-5.37,-6.76,-5.16,-6.16,...,-4.22,-4.24,-4.02,-4.67,-4.7,-5.95,-6.35,-7.13,-7.58,-7.15
1980-01-04,-9.28,-9.26,-9.31,-9.83,-10.9,-9.38,-11.5,-9.82,-10.2,-10.2,...,-10.9,-10.8,-10.3,-11.5,-12.4,-14.4,-14.8,-15.5,-13.9,-12.6
1980-01-05,-16.5,-16.7,-16.8,-17.9,-17.3,-17.4,-17.5,-17.7,-17.5,-17.5,...,-10.9,-11.1,-10.9,-11.8,-11.9,-12.5,-12.9,-14.4,-15.6,-16.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,-9.9,-10.2,-10.3,-11.3,-9.98,-10.4,-10.1,-11.1,-10.5,-10.6,...,-9.85,-9.84,-9.66,-10.5,-6.31,-5.95,-6.76,-9.6,-9.8,-8.66
2018-12-28,-12.5,-12.7,-12.8,-15.5,-13.8,-13.2,-14.1,-14.9,-14.3,-14.1,...,-12.7,-12.4,-12.4,-12,-6.49,-8.55,-9.28,-12.6,-12.9,-10.8
2018-12-29,-6.04,-6.83,-6.78,-6.03,-4.47,-6.67,-4.25,-5.95,-3.89,-5.09,...,-5.59,-5.81,-5.68,-5.38,0.22,-1.44,-2.5,-3.39,-3.23,-2.29


Convert to mm seperately

In [32]:
precipitation_pivoted

seg_nhm,58219,58241,58237,58232,58214,58233,58213,58223,58221,58216,...,58448,58380,58342,58582,58405,58610,58389,58570,58579,58465
time,,,,,,,,,,,,,,,,,,,,,
1980-01-01,0.000754,0.000938,0.000873,0.000344,0.000113,0.000604,5.94e-05,0.000345,5.55e-05,0.000201,...,0.000131,0.000122,0.00011,0.000109,0,0,4.67e-09,2.65e-06,3.24e-06,6.27e-07
1980-01-02,0.000651,0.000967,0.000936,0.000403,0.000193,0.00103,0.000141,0.000282,0.000102,0.000169,...,0.000798,0.000894,0.000836,0.000692,0.000136,1.21e-05,8.81e-06,7.62e-05,8.66e-05,4.76e-05
1980-01-03,0.00444,0.00517,0.00477,0.00105,0.000325,0.00327,6.59e-05,0.00114,4.13e-05,0.000662,...,0.00146,0.00135,0.00152,0.00219,0.000361,0.000158,6.35e-05,7.85e-05,0.000139,0.000268
1980-01-04,0.000467,0.000369,0.00034,0.000198,0.000141,0.000175,7.88e-05,0.000211,7.39e-05,0.00014,...,0.00192,0.00255,0.00239,0.00205,0.000477,0.000411,0.000214,0.0002,0.000142,0.000136
1980-01-05,0.017,0.0176,0.0174,0.0148,0.0123,0.017,0.0119,0.0147,0.0118,0.0134,...,0.0111,0.0115,0.0106,0.00959,0.00608,0.00462,0.0035,0.0034,0.00359,0.00477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,0.000664,0.000655,0.000654,0.000476,0.0004,0.000745,0.000349,0.000438,0.000339,0.000396,...,0.000338,0.00031,0.000311,0.000302,0.000281,0.000455,0.000474,0.000187,0.000196,0.000232
2018-12-28,0.000418,0.00042,0.000401,0.000253,0.000141,0.000312,0.000116,0.00026,0.000116,0.000195,...,0.000216,0.00024,0.000215,0.00024,4.55e-05,2.93e-05,5.02e-05,7.67e-05,5.77e-05,2.64e-05
2018-12-29,0.0132,0.0178,0.017,0.00817,0.00262,0.0149,0.00195,0.00748,0.0019,0.00466,...,0.000144,0.000139,0.000192,0.000225,3.25e-06,5.74e-06,5.49e-06,0,1.36e-07,1.76e-06


In [33]:
# Save the DataFrames to text files with time and separate columns for each ID
precipitation_pivoted.to_csv(os.path.join(output_folder_path, 'Pobs(m).txt'), sep='\t', na_rep='', index_label='time')
temperature_pivoted.to_csv(os.path.join(output_folder_path, 'Tobs.txt'), sep='\t', na_rep='', index_label='time')